In [3]:
import numpy as np
import random
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [23]:
class Cell():
    def __init__(self, cell_id, x, y):
        self.cell_id = cell_id
        self.x = x
        self.y = y
        self.is_occupied = False
        self.next_cell = None
        self.root = self
 
    def get_coordinates(self):
        return (self.x, self.y)

    def get_root(self):
        if self.root == self:
            return self
        else:
            root = self.root
            while root != root.get_root():
                root = root.get_root()
            else:
                return root
   
    def set_root(self, root):
        self.root = root
    
    def set_next(self, other):
        if self.get_next_cell() == self:
            self.next_cell = other
        else:
            cell = self
            path = [other]
            while cell.get_next_cell() != cell:
                path.append(cell)
                cell = cell.get_next_cell()
            else:
                path.append(cell)
            for i in range(len(path) - 1):
                path[-(i + 1)].next_cell = path[-(i+2)]
        
    def get_next_cell(self):
        if self.next_cell:
            return self.next_cell
        else:
            return self
        
    
    def occupy_cell(self):
        self.is_occupied = True
    
    def compress_path(self, root):
        # redirect cell's root to new root
        if self.get_root() != self:
            self.get_root().set_root(root)
        
        # now set original cell's root
        self.set_root(root)
        
        # now set root of all cells that are linked to original cell
        cell = self
        while cell.next_cell:
            if cell.next_cell.get_root() == root:
                break
            cell.next_cell.set_root(root)
            cell = cell.next_cell
        else:
            cell.set_root(root)
            
            
    def get_distance_to_next(self):
        if self.next_cell:
            dx = self.next_cell.x - self.x
            dy = self.next_cell.y - self.y 
            #check for wrapping
            if np.abs(dx) > 1:
                dx = -1 * np.sign(dx) 
            if np.abs(dy) > 1:
                dy = -1 * np.sign(dy) 
        else:
            dx = 0
            dy = 0
            
        return (dx, dy)
    
    def get_spanning_vector2(self):
        cell = self
        dx = 0
        dy = 0
        depth = 0
        while cell.next_cell and cell != self.get_root():
            #print(cell.cell_id, cell.get_root().cell_id, "->", cell.get_next_cell().cell_id, cell.get_next_cell().get_root().cell_id)
            (x, y) = cell.get_distance_to_next()
            dx += x
            dy += y
            cell = cell.get_next_cell()
            depth += 1
            if depth > 100:
                stop
        return (dx, dy)
    
    def get_spanning_vector(self):
        cell = self
        dx = 0
        dy = 0
        depth = 0
        if cell.get_next_cell() != cell.get_root():
            (ax, ay) = cell.get_next_cell().get_spanning_vector()
            (bx, by) = cell.get_distance_to_next()
            (dx, dy) = (ax + bx, ay + by)
        else:
            return cell.get_distance_to_next()
        return (dx, dy)
    
    
    def get_neighbours(self, L):
        xl = self.x - 1 if self.x > 0 else (L - 1)
        yu = self.y + 1 if self.y < (L - 1) else 0
        xr = self.x + 1 if self.x < (L - 1) else 0
        yd = self.y - 1 if self.y > 0 else (L - 1)
        return [(xl, y), (x, yu), (xr, y), (x, yd)]
    
    
    def __eq__(self, other):
        return (self.cell_id == other.cell_id)    
    
    
    def __ne__(self, other):
        return (self.cell_id != other.cell_id)    
    
    def is_equal(self, other):
        return (self.cell_id != other.cell_id) 
    
    

In [24]:
class Cluster():
    def __init__(self, id, cell):
        self.id = id
        self.cells = [cell]
        self.percolates = False
        
    def add_cells(self, cells):
        self.cells += cells
        
    def get_size(self):
        return len(self.cells)
    
    def get_cells(self):
        return self.cells
    
    def set_percolates(self):
        self.percolates = True
        
    def does_percolate(self):
        return self.percolates


class ClusterContainer():
    def __init__(self):
        self.clusters = {}
        
    def merge_clusters(self, id1, id2):
        if self.clusters[id1].get_size() >= self.clusters[id2].get_size():
            self.clusters[id1].add_cells(self.clusters[id2].get_cells())
            self.clusters.pop(id2)
        else:
            self.clusters[id2].add_cells(self.clusters[id1].get_cells())
            self.clusters.pop(id1)
            
    def add_cluster(self, id, cell):
        self.clusters[id] = Cluster(id, cell)
        
    def get_cluster_size(self, id):
        return self.clusters[id].get_size()
    
    def set_percolates(self, id):
        self.clusters[id].set_percolates()
        
    def print_clusters(self):
        for id, cluster in self.clusters.items():
            print(id, cluster.get_size(), cluster.cells)

In [25]:
# some helper functions

def plot_lattice(lattice, N):
    tmp2 = np.full(lattice.shape, 0, dtype='i')
    tmp = np.full(lattice.shape, 0, dtype='i')
    for i in range(N):
        x = int(i % L)
        y = int((i - x) / L)
        if lattice[x][y].is_occupied:
            #print(str(lattice[x][y].get_next_cell().cell_id) + "/" + str(lattice[x][y].get_root().cell_id))
            tmp[x][y] =  lattice[x][y].get_next_cell().cell_id
            tmp2[x][y] = lattice[x][y].get_root().cell_id
    print(tmp)
    print()
    print(tmp2)
    plt.matshow(tmp2)
    plt.show()

def print_cluster(clustersize):
    for cluster, size in clustersize.items():
        if size > 0:
            print(cluster, " ", size)

In [35]:
L = 65
N = L * L
np.random.seed(921)
# create a lattice and fill it with 'empty' cells
lattice = np.ndarray([L, L], dtype = type(Cell))
coordinates = []
random_ids = np.array([i for i in range(N)])
np.random.shuffle(random_ids)
print(random_ids)
for i in range(N):
    x = int(i % L)
    y = int((i - x) / L)
    lattice[x][y] = Cell(i + 1, x, y)
    #lattice[x][y] = Cell(random_ids[i] + 1, x, y)
    coordinates.append((x, y))
    
    
# shuffle coordinates to randomly select cells to be filled:
np.random.shuffle(coordinates)

[ 777  642  480 ... 2273  562 1059]


In [36]:
start = time.time()
clusters = ClusterContainer()
for (x, y) in coordinates:
    lattice[x][y].occupy_cell()
    #print(x, y)
    #plot_lattice(lattice, N)
    clusters.add_cluster(lattice[x][y].cell_id, lattice[x][y])
    
    #plot_lattice(lattice, N)
    
    #check for neighbouring cells:
    nn_coordinates = lattice[x][y].get_neighbours(L)
    
    for (nx, ny) in nn_coordinates:
        if lattice[nx][ny].is_occupied:
           
            r1 = lattice[x][y].get_root()
            id1 = lattice[x][y].cell_id
            size1 = clusters.get_cluster_size(r1.cell_id)
            
            r2 = lattice[nx][ny].get_root()
            id2 = lattice[nx][ny].cell_id
            size2 = clusters.get_cluster_size(r2.cell_id)
            
            if r1 != r2:
                if size1 < size2:
                    lattice[x][y].set_next(lattice[nx][ny])
                    lattice[x][y].compress_path(r2)
                else:
                    lattice[nx][ny].set_next(lattice[x][y])
                    lattice[nx][ny].compress_path(r1)
                #merge the two clusters
                clusters.merge_clusters(r1.cell_id, r2.cell_id)
            else:
                (ax, ay) = lattice[x][y].get_spanning_vector()
                (bx, by) = lattice[nx][ny].get_spanning_vector()
                #print()
                #print("first  cell coordinates:", x, y, "distance to root:",ax, ay, "root",  lattice[x][y].get_root().get_coordinates(), lattice[x][y].get_root().cell_id)
                #print("second cell coordinates:", nx, ny, "distance to root:",bx, by, "root",  lattice[nx][ny].get_root().get_coordinates(), lattice[nx][ny].get_root().cell_id)
                #print(nx, ny, bx, by)
                if np.abs(ax - bx) > 1 or np.abs(ay - by) > 1:
                    #print(ax,"-", bx, "=", ax - bx, "/", ay, by, ay - by)
                    print("PERCOLATION", lattice[x][y].get_root().cell_id, lattice[x][y].cell_id, lattice[nx][ny].cell_id)
                    clusters.set_percolates(r1.cell_id)
                    #plot_lattice(lattice, N)
end = time.time()
print("total time:", end - start)

PERCOLATION 434 3726 3727
PERCOLATION 434 315 316
PERCOLATION 434 655 720
PERCOLATION 434 3822 3757
PERCOLATION 434 137 202
PERCOLATION 434 2716 2717
PERCOLATION 434 525 526
PERCOLATION 434 980 981
PERCOLATION 434 648 713
PERCOLATION 434 648 649
PERCOLATION 434 3406 3407
PERCOLATION 434 578 579
PERCOLATION 434 513 514
PERCOLATION 434 513 448
PERCOLATION 434 3148 3083
PERCOLATION 434 332 397
PERCOLATION 434 201 202
PERCOLATION 434 3821 3822
PERCOLATION 434 3830 3765
PERCOLATION 434 914 849
PERCOLATION 434 3832 3897
PERCOLATION 434 3832 3833
PERCOLATION 434 3832 3767
PERCOLATION 434 584 649
PERCOLATION 434 1504 1505
PERCOLATION 434 1504 1439
PERCOLATION 434 712 777
PERCOLATION 434 712 713
PERCOLATION 434 1707 1642
PERCOLATION 434 1706 1771
PERCOLATION 434 1706 1707
PERCOLATION 434 250 251
PERCOLATION 434 2181 2246
PERCOLATION 434 2181 2182
PERCOLATION 434 2181 2116
PERCOLATION 434 3888 3823
PERCOLATION 434 3665 3600
PERCOLATION 434 2579 2644
PERCOLATION 434 15 4175
PERCOLATION 434 644 57